# Capacitance matrix extraction and LOM analysis

## Prerequisite
You must have a working local installation of Ansys.

## 1. Create the design in Metal

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [ ]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander

In [ ]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

### In this example, the design consists of 4 qubits and 4 CPWs

In [ ]:
# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the defaults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict(
        readout = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        bus1 = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        bus2 = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

RouteMeander.get_template_options(design)

options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        hfss_wire_bonds = True,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

asym = 140
cpw1 = connect('cpw1', 'Q1', 'bus2', 'Q2', 'bus1', '6.0 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'bus1', 'Q2', 'bus2', '6.1 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'bus2', 'Q4', 'bus1', '6.0 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'bus1', 'Q4', 'bus2', '6.1 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()

## 2. Render into Ansys Q3D

In [ ]:
fourq_q3d = design.renderers.q3d

Skip the next cell if Ansys is already open and with a project loaded.

In [ ]:
#fourq_q3d.open_ansys()

**Wait for Ansys to open up completely before proceeding to the next cell.**<br>
If necessary, also close any Ansys popup windows.
If no project is currently open, one can be added via the below cell.

In [ ]:
#fourq_q3d.new_ansys_project()

In [ ]:
fourq_q3d.connect_ansys()
fourq_q3d.add_q3d_design("Q3dMetalDesign")

Render a single qubit with 2 endcaps

In [ ]:
fourq_q3d.render_design(['Q1'], [('Q1', 'readout'), ('Q1', 'bus1'), ('Q1', 'bus2')])

## 3. Create a new solution setup in Ansys Q3D and obtain the capacitance matrix

In [ ]:
#     Add a solution setup:
fourq_q3d.add_q3d_setup()
#     Example of how to change defaults (just an example. We will not use this setup):
# fourq_q3d.add_q3d_setup(name = 'Setup2', max_passes = 20)

Optional keyword parameters and their defaults, listed below, represent parameters in this Q3D solution setup. Running fourq_q3d.add_q3d_setup() is equivalent to going to the Project Manager panel in Ansys, right clicking on Analysis within the active Q3D design, selecting "Add Solution Setup...", and choosing/entering default values in the resulting popup window. To easily keep track of different solution setups, give each of them a different name, otherwise, Ansys will simply append numbers to the end of duplicate names. <br><br>
freq_ghz: float = 5. <br>
name: str = "Setup" <br>
save_fields: bool = False <br>
enabled: bool = True <br>
max_passes: int = 15 <br>
min_passes: int = 2 <br>
min_converged_passes: int = 2 <br>
percent_error: float = 0.5 <br>
percent_refinement: int = 30 <br>
auto_increase_solution_order: bool = True <br>
solution_order: str = 'High' <br>
solver_type: str = 'Iterative <br><br>
These defaults can be modified by including the keywords and their new values inside the parentheses. For example, to increase the maximum number of passes from the default value of 15 up to 20 while leaving all else unchanged, replace the cell above with the following: <br><br>
fourq_q3d.add_q3d_setup(max_passes = 20)

In [ ]:
# Analyze said solution setup.
fourq_q3d.analyze_setup("Setup")

In [ ]:
# Using the analysis results, get its capacitance matrix as a dataframe.
fourq_q3d.get_capacitance_matrix()

The optional keyword parameters of get_capacitance_matrix are given below: <br><br>
variation: str = '' <br>
solution_kind: str = 'LastAdaptive' <br>
pass_number: int = 3

In [ ]:
# Run lumped oscillator model (LOM) simulations and save results in dict_lom.b
dict_lom = fourq_q3d.lumped_oscillator_vs_passes(12.31, 2, 3, 7.0, [6.0, 6.2], 9)
dict_lom

Using capacitance matrices obtained from each pass, save the many parameters of the Hamiltonian of the system. lumped_oscillator_vs_passes takes in the following parameters: <br><br>
Lj_nH: float - the Josephson junction inductance, influences your qubit's frequency.<br> 
Cj_fF: float - the Josephson junction capacitance, influences your qubit's frequency and anharmonicity.<br>
N: int - the number of connections in your simulation, eg. 1 readout and 2 busses -> 3 <br>
fr: Union[list, float] - the frequency of your readout resonator. <br>
fb: Union[list, float] - the frequencies of your busses (use an empty list if no busses present).<br>
maxPass: int - the total number of passes your simulation took to converge.<br>
variation: str = '' - if variations of the simulation were run, picking which variation to do the analysis on.<br>
g_scale: float = 1

dist_analysis allows one to calculate dissipation and other properties. It is used for the convergence plots below.

In [ ]:
fourq_q3d.plot_convergence_main(dict_lom);
fourq_q3d.plot_convergence_chi(dict_lom)

In [ ]:
fourq_q3d.disconnect_ansys()

In [ ]:
gui.main_window.close()